# Projekt 2 -- małpia układanka -- Kacper Jabłonka 

## 1. Opis problemu

"Małpia układanka", czyli Edge-matching puzzle, to rodzaj zagadki logicznej, w której gracze muszą ułożyć serie kafelków (lub klocków) na planszy tak, aby kolory na sąsiadujących krawędziach pasowały do siebie. Każdy kafelek ma kolor na każdym z czterech boków i celem jest umieszczenie kafelków na planszy w taki sposób, aby kolory na stykających się krawędziach były takie same.

Każdy kafelek może być obrócony o 0, 90, 180 lub 270 stopni.

Aby rozwiązać ten problem, modelujemy go jako problem całkowitoliczbowego programowania liniowego. Początkowo tworzymy cztery tablice: CT, CB, CL, CR. Dla każdej z nich, CT[t][a][l] przyjmuje wartość 1 jeśli t-ty kafelek po obróceniu o a stopni ma na górnej krawędzi (odpowiednio: dolnej, lewej, prawej dla CB, CL, CR) kolor l. W przeciwnym wypadku, przyjmuje wartość 0.

## 2. Przygotowanie danych

In [2]:
# Definiujemy klocki jako n^2-elementową listę; kolejne cyfry odpowiadają kolorom (góra, prawo, dół, lewo)
tiles = [
    [1, 2, 3, 5],
    [5, 4, 6, 2],
    [1, 5, 3, 4],
    [2, 1, 6, 5],
    [3, 1, 6, 2],
    [6, 5, 4, 1],
    [3, 4, 2, 5],
    [6, 2, 3, 4],
    [6, 3, 4, 5],
    [4, 1, 2, 3],
    [2, 5, 4, 1],
    [3, 1, 6, 5],
    [4, 6, 1, 2],
    [2, 3, 5, 6],
    [4, 1, 6, 3],
    [6, 3, 2, 1]
]

# Zmniejszamy o 1 wartość każdego koloru, aby zbiór kolorów mógł być indeksem
tiles = [[c - 1 for c in tile] for tile in tiles]

In [3]:
# Generator
def generate_color_matrix(tiles, color_count=6, rotation_count=4):
    """
    Generuje macierze kolorów dla układanki, które później posłużą jako input do problemu optymalizacyjnego.
    """
    tile_count = len(tiles)

    C_matrices = [[[[0]*color_count for _ in range(rotation_count)] for _ in range(tile_count)] for _ in range(4)]
    CT, CR, CB, CL = C_matrices
    
    for i, tile in enumerate(tiles):
        for j, color in enumerate(tile):
            CT[i][0][tile[0]] = CR[i][1][tile[3]] = CB[i][2][tile[2]] = CL[i][3][tile[1]] = 1
            CR[i][0][tile[1]] = CT[i][1][tile[0]] = CL[i][2][tile[3]] = CB[i][3][tile[2]] = 1
            CB[i][0][tile[2]] = CL[i][1][tile[1]] = CT[i][2][tile[0]] = CR[i][3][tile[3]] = 1
            CL[i][0][tile[3]] = CB[i][1][tile[2]] = CR[i][2][tile[1]] = CT[i][3][tile[0]] = 1
            
    return CT, CR, CB, CL

CT, CR, CB, CL = generate_color_matrix(tiles)

Ta funkcja generuje cztery macierze kolorów, które reprezentują kolory krawędzi każdego kafelka dla każdej możliwej rotacji. Wykorzystywane są one później do określenia, które krawędzie kafelków są dopasowane.

Macierze CT, CR, CB, CL to cztery trójwymiarowe macierze, które mają tyle samo elementów, ile jest kafelków w układance, i każda z nich ma tyle samo elementów, ile jest możliwych rotacji. Każdy element trójwymiarowej macierzy jest listą z tyloma zerami, ile jest możliwych kolorów.

W podwójnej pętli for funkcja przechodzi przez każdy kafelek (który jest reprezentowany przez listę kolorów, jeden kolor dla każdej krawędzi) i ustawia wartość 1 w odpowiednich macierzach dla kolorów każdej krawędzi.

Dla przykładu, dla kafelka `i` i koloru `0` (górnego koloru dla danego kafelka), `CT[i][0][tile[0]] = 1`. To oznacza, że po obróceniu kafelka `i` o `0` stopni, górna krawędź (CT) ma kolor `tile[0]`.

W każdym z czterech bloków instrukcji w pętli, dla danego kafelka i dla każdej możliwej rotacji ustawiana jest wartość 1 w odpowiedniej pozycji macierzy, odpowiadającej danemu kolorowi krawędzi. Dzięki temu dla każdego kafelka i dla każdej możliwej rotacji znamy kolor każdej krawędzi.

Tak przygotowane macierze są potem wykorzystywane przy formułowaniu problemu jako problemu programowania liniowego.


## 3. Sformułowanie problemu liniowego



Używamy klasy `MixedIntegerLinearProblem` do stworzenia problemu całkowitoliczbowego programowania liniowego, definiując trzy zmienne binarne decyzyjne:

1. $x_{t,r,c,a}$ - przyjmuje wartość 1, jeżeli t-ty kafelek jest na pozycji $(r,c)$ i obrócony o $a$ stopni. W przeciwnym wypadku jest równe 0.
2. $h_{r,c}$ - przyjmuje wartość 1, jeżeli prawa krawędź na pozycji $(r,c)$ jest niedopasowana. W przeciwnym wypadku jest równe 0.
3. $v_{r,c}$ - przyjmuje wartość 1, jeżeli dolna krawędź na pozycji $(r,c)$ jest niedopasowana. W przeciwnym wypadku jest równe 0.

Wprowadzamy trzy niezbędne ograniczenia:

1. Każdy klocek jest na jednym miejscu. To oznacza, że suma dla wszystkich możliwych pozycji i orientacji każdego klocka musi być równa 1. Możemy to zapisać następująco:

   $
   \sum_{{r=0}}^{{n-1}} \sum_{{c=0}}^{{n-1}} \sum_{{a=0}}^{{3}} x_{trca} = 1 \quad \forall t \in \{0, 1, ..., T-1\}
   $

   gdzie $x_{trca}$ to zmienna binarna wskazująca, czy klocek $t$ jest na pozycji $(r, c)$ i obrócony o $a \times 90$ stopni, $n$ to rozmiar siatki, a $T$ to liczba klocków.

2. W każdym miejscu jest jeden klocek. Oznacza to, że suma dla wszystkich możliwych klocków i orientacji na każdej pozycji musi być równa 1. Możemy to zapisać tak:

   $
   \sum_{{t=0}}^{{T-1}} \sum_{{a=0}}^{{3}} x_{trca} = 1 \quad \forall r \in \{0, 1, ..., n-1\}, c \in \{0, 1, ..., n-1\}
   $

3. Sprawdzenie dopasowania kolorów. Dla każdej pary klocków sąsiadujących bokami (poziomo i pionowo), kolory muszą się zgadzać. Dla uproszczenia zapisu, zaprezentujmy tylko wersję dla pary sąsiadującej poziomo:

   $
   \sum_{{t=0}}^{{T-1}} \sum_{{a=0}}^{{3}} CR_{tal} x_{trca} - \sum_{{t=0}}^{{T-1}} \sum_{{a=0}}^{{3}} CL_{tal} x_{trc(a+1)} \leq h_{rc} \quad \forall r \in \{0, 1, ..., n-1\}, c \in \{0, 1, ..., n-2\}, l \in \{0, 1, ..., L-1\}
   $

   $
   -\sum_{{t=0}}^{{T-1}} \sum_{{a=0}}^{{3}} CR_{tal} x_{trca} + \sum_{{t=0}}^{{T-1}} \sum_{{a=0}}^{{3}} CL_{tal} x_{trc(a+1)} \leq h_{rc} \quad \forall r \in \{0, 1, ..., n-1\}, c \in \{0, 1, ..., n-2\}, l \in \{0, 1, ..., L-1\}
   $

   Tutaj $CR_{tal}$ i $CL_{tal}$ są binarnymi zmiennymi wskazującymi, czy klocek $t$ obrócony o $a \times 90$ stopni ma kolor $l$ na swojej prawej (CR) lub lewej (CL) krawędzi. $L$ jest liczbą kolorów, a $h_{rc}$ jest zmienną binarną, która przyjmuje wartość 1, jeżeli kolory na prawej krawędzi klocka w pozycji $(r, c)$ i na lewej krawędzi klocka na pozycji $(r, c+1)$ są różne, oraz 0 w przeciwnym przypadku.

   Analogicznie zapisujemy dla pary sąsiadującej pionowo, tylko używamy zmiennych $CB_{tal}$ i $CT_{tal}$ (dla dolnej i górnej krawędzi), a ograniczenie jest dla $v_{rc}$, co oznacza różnicę kolorów między dolną krawędzią klocka w pozycji $(r, c)$ i górną krawędzią klocka na pozycji $(r+1, c)$.

Celem jest zminimalizowanie liczby przypadków, gdy krawędzie sąsiadujących kafelków mają różne kolory, co jest wyrażone jako minimalizacja sumy $h_{rc}$ i $v_{rc}$ dla wszystkich par sąsiadujących kafelków, tzn. funkcja celu ma postać:

$\sum_{r=0}^{n-1}\sum_{c=0}^{n-2} h_{rc} + \sum_{r=0}^{n-2}\sum_{c=0}^{n-1} v_{rc}$


In [4]:
# Formulacja problemu
def solve_puzzle(tiles, CT, CR, CB, CL, color_count=6, rotation_count=4, solver='GLPK'):
    """
    Rozwiązuje problem układanki, przyjmując odpowiednio zestaw klocków 
    oraz wygenerowane wcześniej macierze problemu. 
    """
    tile_count = len(tiles)
    grid_size = int(sqrt(tile_count))

    P = MixedIntegerLinearProgram(maximization=False, solver=solver)
    x = P.new_variable(binary=True, name='x')
    h = P.new_variable(binary=True, name='h')
    v = P.new_variable(binary=True, name='v')

    # Każdy klocek jest na dokładnie jednym miejscu
    for t in range(tile_count):
        P.add_constraint(
            P.sum(x[(t,r,c,a)] for r in range(grid_size) for c in range(grid_size) for a in range(rotation_count)) == 1)

    # W każdym miejscu jest dokładnie jeden klocek
    for r in range(grid_size):
        for c in range(grid_size):
            P.add_constraint(
                P.sum(x[(t,r,c,a)] for t in range(tile_count) for a in range(rotation_count)) == 1)

    # Sprawdzanie dopasowania kolorów
    for r in range(grid_size):
        for c in range(grid_size - 1):
            for l in range(color_count):
                P.add_constraint(
                    P.sum(CR[t][a][l]*x[(t,r,c,a)] for t in range(tile_count) for a in range(rotation_count))
                    -P.sum(CL[t][a][l]*x[(t,r,c+1,a)] for t in range(tile_count) for a in range(rotation_count))
                    <= h[(r, c)])
                P.add_constraint(
                    -P.sum(CR[t][a][l]*x[(t,r,c,a)] for t in range(tile_count) for a in range(rotation_count))
                    +P.sum(CL[t][a][l]*x[(t,r,c+1,a)] for t in range(tile_count) for a in range(rotation_count))
                    <= h[(r, c)])
    
    for r in range(grid_size - 1):
        for c in range(grid_size):
            for l in range(color_count):
                P.add_constraint(
                    P.sum(CB[t][a][l]*x[(t,r,c,a)] for t in range(tile_count) for a in range(rotation_count))
                    -P.sum(CT[t][a][l]*x[(t,r+1,c,a)] for t in range(tile_count) for a in range(rotation_count))
                    <= v[(r,c)])
                P.add_constraint(
                    -P.sum(CB[t][a][l]*x[(t,r,c,a)] for t in range(tile_count) for a in range(rotation_count))
                    +P.sum(CT[t][a][l]*x[(t,r+1,c,a)] for t in range(tile_count) for a in range(rotation_count))
                    <= v[(r,c)])
                
    # Ustalamy cel jako minimalizację niespasowanych krawędzi
    P.set_objective(
        P.sum(h[(r,c)] for r in range(grid_size) for c in range(grid_size - 1))
        +P.sum(v[(r,c)] for r in range(grid_size - 1) for c in range(grid_size)))

    # Rozwiązujemy problem
    P.solve()

    # Przygotowujemy wynik
    result = {variable: value for variable, value in P.get_values(x).items() if value == 1.0}
    
    return P, result

P, result = solve_puzzle(tiles, CT, CR, CB, CL)

Przeanalizujmy działanie pętli, która sprawdza zgodność kolejnych klocków:

1. W dwóch zewnętrznych pętlach iterujemy przez wszystkie klocki w siatce z wyjątkiem prawej krawędzi (dla `c`) w pierwszym bloku i dolnej krawędzi (dla `r`) w drugim bloku. Dla każdej pozycji (r, c) w siatce sprawdzamy dopasowanie kolorów.

2. Wewnętrzna pętla `for l in range(color_count):` iteruje przez wszystkie kolory.

3. Następnie dodajemy dwa ograniczenia dla każdej pary klocków sąsiadujących bokami (poziomo i pionowo). Pierwsze ograniczenie sprawdza, czy prawy kolor klocka (dla kolumny `c`) jest taki sam jak lewy kolor klocka na prawo od niego (dla kolumny `c+1`). Podobnie, drugie ograniczenie sprawdza, czy dolny kolor klocka (dla wiersza `r`) jest taki sam jak górny kolor klocka poniżej niego (dla wiersza `r+1`).

4. Zmienne `h[(r, c)]` i `v[(r,c)]` służą do zapewnienia, że rozwiązanie jest dozwolone, jeżeli kolory się nie zgadzają. Są one równe 1, jeśli kolory są różne, i 0, jeśli są takie same. 

5. Dzięki temu układ ograniczeń, program rozwiązuje problem tak, aby zminimalizować liczbę niespasowanych krawędzi (jak to określono w funkcji celu), zmuszając do dopasowania jak największej liczby kolorów.

Tym samym mamy już funkcję kosztu, którą minimalizujemy. 

## 4. Rozwiązanie

Gdy problem zostaje pomyślnie rozwiązany, rezultat jest reprezentowany jako słownik. Klucze tego słownika, będące krotkami w formie $(t,r,c,a)$, wskazują na konkretne układy układanki: kafelek o indeksie $t$, umieszczony na współrzędnych $(r,c)$ i obrócony o $a$ stopni. Wartość przypisana do takiego klucza wynosi 1, sugerując, że dany układ jest obecny w rozwiązaniu. Wszystkie pozostałe układy, które nie występują w rozwiązaniu, nie są uwzględniane w słowniku. W efekcie, dostajemy bezpośrednie informacje na temat rozmieszczenia i orientacji każdego kafelka w ostatecznym rozwiązaniu.

In [5]:
result

{(0, 0, 1, 0): 1.0,
 (1, 1, 3, 1): 1.0,
 (2, 1, 2, 1): 1.0,
 (3, 2, 0, 1): 1.0,
 (4, 2, 2, 1): 1.0,
 (5, 2, 1, 1): 1.0,
 (6, 3, 3, 0): 1.0,
 (7, 2, 3, 1): 1.0,
 (8, 3, 2, 1): 1.0,
 (9, 1, 0, 0): 1.0,
 (10, 0, 0, 2): 1.0,
 (11, 1, 1, 1): 1.0,
 (12, 0, 2, 0): 1.0,
 (13, 0, 3, 0): 1.0,
 (14, 3, 1, 1): 1.0,
 (15, 3, 0, 3): 1.0}